In [15]:
import os

import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from statsmodels.nonparametric.smoothers_lowess import lowess

In [16]:
os.chdir(os.path.join("/", "home", "walkerdavis", "projects", "mpsc"))
DATA_ROOT = os.path.join("data", "processed")

np.random.seed(42)

In [17]:
class GrowthDataTransformation:
    def __init__(self):
        self.consolidated_data = None
        self.growth_models = {}

    def load_and_process_demographics(self, demographics_path):
        """Load and process demographics data"""
        demographics = pd.read_csv(demographics_path)

        demographics["GA"] = pd.to_numeric(
            demographics["GA"].str.extract(r"(\d+)")[0], errors="coerce"
        )

        demographics["DOB"] = pd.to_datetime(demographics["DOB"])
        demographics["DataStart"] = pd.to_datetime(demographics["DataStart"])
        demographics["DataEnd"] = pd.to_datetime(demographics["DataEnd"])
        demographics["FluidStart"] = pd.to_datetime(demographics["FluidStart"])
        demographics["FluidEnd"] = pd.to_datetime(demographics["FluidEnd"])

        demographics = demographics[
            (demographics["FluidStart"] < pd.to_datetime("2200-01-01"))
            & (demographics["GA"].notna())
            & (demographics["GA"] > 20)
            & (demographics["BW"].notna())
            & (demographics["BW"] > 200)
        ]

        return demographics

    def load_and_process_weight_data(self, weight_path):
        """Load and process weight data"""
        weight_data = pd.read_csv(weight_path)

        weight_data["Value"] = weight_data["Value"] * 28.35
        weight_data["OBX"] = "WeightG"
        weight_data["DateTime"] = pd.to_datetime(weight_data["DateTime"])

        weight_data = weight_data.dropna(subset=["Value"])

        return weight_data[["ID", "DateTime", "OBX", "Value"]]

    def create_growth_tibble(self, demographics, weight_data):
        """Join demographics and weight data: compute additional variables"""
        growth_data = demographics[["ID", "DOB", "GA", "BW", "Sex"]].merge(
            weight_data, on="ID", how="inner"
        )

        growth_data["DayFromBirthNumeric"] = (
            growth_data["DateTime"] - growth_data["DOB"]
        ).dt.days.astype(float)

        growth_data["PMADays"] = (
            7 * growth_data["GA"] + growth_data["DayFromBirthNumeric"]
        ).astype(int)

        growth_data["PMAWeeks"] = (growth_data["PMADays"] / 7).astype(int)

        return growth_data

    def fit_growth_models(self, growth_data, frac=0.3):
        patient_counts = growth_data.groupby("ID").size()
        valid_patients = patient_counts[patient_counts >= 3].index

        growth_data = growth_data[growth_data["ID"].isin(valid_patients)]

        models = {}
        smoothed_data = []

        print(f"Num Valid Patients: {len(valid_patients)}")

        for patient_id in valid_patients:
            patient_data = growth_data[growth_data["ID"] == patient_id].copy()
            patient_data = patient_data.sort_values("DayFromBirthNumeric")

            x = patient_data["DayFromBirthNumeric"].values
            y = patient_data["Value"].values

            if len(np.unique(x)) < 3:
                continue

            try:
                """
                This line currently uses past values, which introduces data leakage into
                any predictor that uses the smoothed weight. Use a simple moving
                average instead.
                """
                fitted = lowess(y, x, frac=frac, return_sorted=False)

                # fitted = pd.Series(y).expanding(min_periods=1).mean().values

                interp_func = interp1d(
                    x,
                    fitted,
                    kind="linear",
                    bounds_error=False,
                    fill_value="extrapolate",
                )

                models[patient_id] = interp_func

                patient_data["fitted"] = interp_func(x)
                smoothed_data.append(patient_data)

            except Exception as e:
                print(f"Error fitting LOESS for patient {patient_id}: {e}")
                continue

        self.growth_models = models

        smoothed_data = pd.concat(smoothed_data, axis=0).reset_index(drop=True)

        return smoothed_data

    def load_and_process_energy_data(self, energy_path):
        energy_data = pd.read_csv(energy_path)
        energy_data["DateTime"] = pd.to_datetime(energy_data["DateTime"])
        energy_data["StartDate"] = energy_data["DateTime"].dt.date

        energy_data["Parenteral"] = energy_data["TreatmentRoute"].isin(["IV"])

        daily_energy = (
            energy_data.groupby(["ID", "StartDate", "Parenteral"])["Value"]
            .sum()
            .reset_index()
        )
        daily_energy.columns = ["ID", "StartDate", "Parenteral", "DailyEnergy"]

        daily_energy_pivot = daily_energy.pivot_table(
            index=["ID", "StartDate"],
            columns="Parenteral",
            values="DailyEnergy",
            fill_value=0,
        ).reset_index()

        daily_energy_pivot.columns = [
            "ID",
            "StartDate",
            "DailyEnergyEnteral",
            "DailyEnergyParenteral",
        ]
        daily_energy_pivot["DailyEnergy"] = (
            daily_energy_pivot["DailyEnergyEnteral"]
            + daily_energy_pivot["DailyEnergyParenteral"]
        )

        return daily_energy_pivot

    def load_and_process_pulse_data(self, pulse_path):
        pulse_data = pd.read_csv(pulse_path)
        pulse_data["DateTime"] = pd.to_datetime(pulse_data["DateTime"])
        pulse_data["StartDate"] = pulse_data["DateTime"].dt.date

        daily_pulse = (
            pulse_data.groupby(["ID", "StartDate"])["Value"].mean().reset_index()
        )
        daily_pulse.columns = ["ID", "StartDate", "MeanDailyPulse"]

        return daily_pulse

    def load_and_process_spo2_data(self, spo2_path):
        spo2_data = pd.read_csv(spo2_path)
        spo2_data["DateTime"] = pd.to_datetime(spo2_data["DateTime"])
        spo2_data["StartDate"] = spo2_data["DateTime"].dt.date

        daily_spo2 = (
            spo2_data.groupby(["ID", "StartDate"])["Value"].mean().reset_index()
        )
        daily_spo2.columns = ["ID", "StartDate", "MeanDailySPO2"]

        return daily_spo2

    def load_and_process_stool_weight_data(self, stool_path):
        stool_data = pd.read_csv(stool_path)
        stool_data["DateTime"] = pd.to_datetime(stool_data["DateTime"])
        stool_data["StartDate"] = stool_data["DateTime"].dt.date

        daily_sum_stool_weight = (
            stool_data.groupby(["ID", "StartDate"])["Value"].sum().reset_index()
        )
        daily_sum_stool_weight.columns = ["ID", "StartDate", "SumDailyStoolWeight"]

        return daily_sum_stool_weight

    def load_and_process_bp_data(self, bp_path: str) -> pd.DataFrame:
        bp_data = pd.read_csv(bp_path)
        bp_data["DateTime"] = pd.to_datetime(bp_data["DateTime"])
        bp_data["StartDate"] = bp_data["DateTime"].dt.date

        bp_data[["systolic_pb", "diastolic_bp"]] = (
            bp_data["Value"].str.split("/", expand=True).astype(float)
        )

        daily_bp = (
            bp_data.groupby(["ID", "StartDate"])[["systolic_pb", "diastolic_bp"]]
            .mean()
            .reset_index()
        )
        daily_bp.columns = [
            "ID",
            "StartDate",
            "MeanDailySystolicBP",
            "MeanDailyDiastolicBP",
        ]

        return daily_bp

    def load_and_process_resp_data(self, resp_path):
        resp_data = pd.read_csv(resp_path)
        resp_data["DateTime"] = pd.to_datetime(resp_data["DateTime"])
        resp_data["StartDate"] = resp_data["DateTime"].dt.date

        daily_resp = (
            resp_data.groupby(["ID", "StartDate"])["Value"].mean().reset_index()
        )
        daily_resp.columns = ["ID", "StartDate", "MeanDailyResp"]

        return daily_resp

    def load_and_process_io_data(self, io_path):
        io_data = pd.read_csv(io_path)
        io_data["StartDate"] = pd.to_datetime(io_data["StartDate"]).dt.date

        io_pivot = io_data.pivot_table(
            index=["ID", "StartDate"], columns="OBX", values="Value", fill_value=0
        ).reset_index()

        return io_pivot

    def predict_smoothed_weight(self, patient_id, day_from_birth):
        if patient_id in self.growth_models:
            return self.growth_models[patient_id](day_from_birth)
        return np.nan

    def calculate_weight_derivatives(self, patient_id, day_from_birth):
        if patient_id not in self.growth_models:
            return np.nan, np.nan

        model = self.growth_models[patient_id]

        try:
            current_weight = model(day_from_birth)
            next_day_weight = model(day_from_birth + 1)
            daily_gain = next_day_weight - current_weight

        except Exception:
            daily_gain = np.nan

        try:
            week_later_weight = model(day_from_birth + 1)
            week_earlier_weight = model(day_from_birth - 6)
            weekly_gain = (week_later_weight - week_earlier_weight) / 7
        except Exception:
            weekly_gain = np.nan

        return daily_gain, weekly_gain

    def calculate_rolling_statistics(
        self, df, patient_col, date_col, value_col, window=7
    ):
        df = df.sort_values([patient_col, date_col])

        df[f"{value_col}_rolling_mean"] = df.groupby(patient_col)[value_col].transform(
            lambda x: x.rolling(window=window, min_periods=1).mean()
        )
        df[f"{value_col}_rolling_std"] = df.groupby(patient_col)[value_col].transform(
            lambda x: x.rolling(window=window, min_periods=1).std()
        )

        return df

    def consolidate_all_data(
        self,
        demographics,
        energy_data,
        pulse_data,
        io_data,
        spo2_data,
        resp_data,
        stool_data,
        bp_data,
    ):
        consolidated = demographics[["ID", "DOB", "GA", "BW", "Sex"]].copy()
        consolidated = consolidated.merge(energy_data, on="ID", how="inner")
        consolidated = consolidated.merge(
            pulse_data, on=["ID", "StartDate"], how="inner"
        )
        consolidated = consolidated.merge(io_data, on=["ID", "StartDate"], how="inner")
        consolidated = consolidated.merge(
            spo2_data, on=["ID", "StartDate"], how="inner"
        )
        consolidated = consolidated.merge(
            resp_data, on=["ID", "StartDate"], how="inner"
        )
        consolidated = consolidated.merge(
            stool_data, on=["ID", "StartDate"], how="inner"
        )
        consolidated = consolidated.merge(bp_data, on=["ID", "StartDate"], how="inner")

        consolidated["DayFromBirthNumeric"] = (
            pd.to_datetime(consolidated["StartDate"]) - consolidated["DOB"]
        ).dt.days.astype(float)

        consolidated["PMADays"] = (
            7 * consolidated["GA"] + consolidated["DayFromBirthNumeric"]
        ).astype(int)

        consolidated["SexMInt"] = (consolidated["Sex"] == "M").astype(int)

        consolidated["SmoothedWeightG"] = consolidated.apply(
            lambda row: self.predict_smoothed_weight(
                row["ID"], row["DayFromBirthNumeric"]
            ),
            axis=1,
        )

        weight_gains = consolidated.apply(
            lambda row: self.calculate_weight_derivatives(
                row["ID"], row["DayFromBirthNumeric"]
            ),
            axis=1,
        )

        consolidated["DailyWeightGainGPerDay"] = [x[0] for x in weight_gains]
        consolidated["WeeklyWeightGainGPerDay"] = [x[1] for x in weight_gains]

        consolidated["DailyEnergyPerKG"] = (
            consolidated["DailyEnergy"] * 1000 / consolidated["SmoothedWeightG"]
        )
        consolidated["DailyEnergyParenteralPerKG"] = (
            consolidated["DailyEnergyParenteral"]
            * 1000
            / consolidated["SmoothedWeightG"]
        )
        consolidated["DailyWeightGainGPerKGPerDay"] = (
            consolidated["DailyWeightGainGPerDay"]
            * 1000
            / consolidated["SmoothedWeightG"]
        )
        consolidated["WeeklyWeightGainGPerKGPerDay"] = (
            consolidated["WeeklyWeightGainGPerDay"]
            * 1000
            / consolidated["SmoothedWeightG"]
        )
        consolidated["MeanDailyPulsePerKG"] = (
            consolidated["MeanDailyPulse"] * 1000 / consolidated["SmoothedWeightG"]
        )

        if "DailyFluidIntake" in consolidated.columns:
            consolidated["DailyFluidIntakePerKG"] = (
                consolidated["DailyFluidIntake"]
                * 1000
                / consolidated["SmoothedWeightG"]
            )
        if "DailyFluidOutput" in consolidated.columns:
            consolidated["DailyFluidOutputPerKG"] = (
                consolidated["DailyFluidOutput"]
                * 1000
                / consolidated["SmoothedWeightG"]
            )

        if "DailyFluidIntake" in consolidated.columns:
            consolidated = self.calculate_rolling_statistics(
                consolidated, "ID", "StartDate", "DailyFluidIntake"
            )
        if "DailyFluidOutput" in consolidated.columns:
            consolidated = self.calculate_rolling_statistics(
                consolidated, "ID", "StartDate", "DailyFluidOutput"
            )

        consolidated["GrowthStage"] = np.where(
            consolidated["PMADays"] < 184,
            1,
            np.where(consolidated["PMADays"] < 237, 2, 3),
        )

        consolidated = consolidated[consolidated["PMADays"] < 280]

        patient_ids = consolidated["ID"].unique()
        train_ids = np.random.choice(
            patient_ids, size=int(len(patient_ids) * 0.5), replace=False
        )
        consolidated["Train"] = consolidated["ID"].isin(train_ids)

        self.consolidated_data = consolidated
        return consolidated

    def get_normal_weekly_growth(self, ga, week_from_birth, weight_g):
        pma = ga + week_from_birth - 1

        if week_from_birth == 1:
            return 0
        elif pma > 34:
            return 17 * weight_g / 1000
        else:
            return 30

    def add_normal_growth_comparison(self):
        if self.consolidated_data is None:
            raise ValueError("No consolidated data available.")

        self.consolidated_data["WeekFromBirth"] = (
            self.consolidated_data["DayFromBirthNumeric"] / 7
        ).astype(int) + 1

        self.consolidated_data["NormalWeeklyGrowthGPerDay"] = (
            self.consolidated_data.apply(
                lambda row: self.get_normal_weekly_growth(
                    row["GA"], row["WeekFromBirth"], row["SmoothedWeightG"]
                ),
                axis=1,
            )
        )

        return self.consolidated_data

    def create_feature_list(self):
        base_features = [
            "GA",
            "BW",
            "DailyEnergy",
            "DailyEnergyParenteral",
            "DayFromBirthNumeric",
            "SmoothedWeightG",
            "MeanDailyPulse",
            "PMADays",
            "SexMInt",
            "DailyEnergyPerKG",
            "DailyEnergyParenteralPerKG",
            "MeanDailyPulsePerKG",
            "MeanDailySystolicBP",
            "MeanDailyDiastolicBP",
            "SumDailyStoolWeight",
            "MeanDailyResp",
            "MeanDailySPO2",
        ]

        if "DailyFluidIntake" in self.consolidated_data.columns:
            base_features.extend(
                [
                    "DailyFluidIntake",
                    "DailyFluidIntakePerKG",
                    "DailyFluidIntake_rolling_mean",
                    "DailyFluidIntake_rolling_std",
                ]
            )

        if "DailyFluidOutput" in self.consolidated_data.columns:
            base_features.extend(
                [
                    "DailyFluidOutput",
                    "DailyFluidOutputPerKG",
                    "DailyFluidOutput_rolling_mean",
                    "DailyFluidOutput_rolling_std",
                ]
            )

        extended_features = base_features.copy()
        for feature in base_features:
            if feature in self.consolidated_data.columns:
                extended_features.append(f"{feature}_squared")
                extended_features.append(f"{feature}_sqrt")

        return extended_features

    def create_polynomial_features(self):
        if self.consolidated_data is None:
            raise ValueError("No consolidated data available.")

        base_features = self.create_feature_list()

        for feature in base_features:
            if feature in self.consolidated_data.columns:
                self.consolidated_data[f"{feature}_squared"] = (
                    self.consolidated_data[feature] ** 2
                )

                self.consolidated_data[f"{feature}_sqrt"] = np.where(
                    self.consolidated_data[feature] >= 0,
                    self.consolidated_data[feature] ** 0.5,
                    np.nan,
                )

        return self.consolidated_data


In [18]:
analysis = GrowthDataTransformation()

demographics = analysis.load_and_process_demographics(
    os.path.join(DATA_ROOT, "Demographics.csv")
)
weight_data = analysis.load_and_process_weight_data(
    os.path.join(DATA_ROOT, "WeightObservations.csv")
)

growth_data = analysis.create_growth_tibble(demographics, weight_data)
smoothed_growth = analysis.fit_growth_models(growth_data)

energy_data = analysis.load_and_process_energy_data(
    os.path.join(DATA_ROOT, "CalculatedEnergyObservations.csv")
)

pulse_data = analysis.load_and_process_pulse_data(
    os.path.join(DATA_ROOT, "PulseObservations.csv")
)

spo2_data = analysis.load_and_process_spo2_data(os.path.join(DATA_ROOT, "SPO2Obs.csv"))

resp_data = analysis.load_and_process_resp_data(os.path.join(DATA_ROOT, "RespObs.csv"))


io_data = analysis.load_and_process_io_data(os.path.join(DATA_ROOT, "DailyIO.csv"))

stool_data = analysis.load_and_process_stool_weight_data(
    os.path.join(DATA_ROOT, "StoolWeightObservations.csv")
)

bp_data = analysis.load_and_process_bp_data(
    os.path.join(DATA_ROOT, "BPObservations.csv")
)

Num Valid Patients: 835


/home/walkerdavis/projects/mpsc/.venv/lib/python3.12/site-packages/statsmodels/nonparametric/smoothers_lowess.py:226: RuntimeWarning: invalid value encountered in divide
  res, _ = _lowess(y, x, x, np.ones_like(x),
/home/walkerdavis/projects/mpsc/.venv/lib/python3.12/site-packages/scipy/interpolate/_interpolate.py:497: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/home/walkerdavis/projects/mpsc/.venv/lib/python3.12/site-packages/scipy/interpolate/_interpolate.py:497: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/home/walkerdavis/projects/mpsc/.venv/lib/python3.12/site-packages/scipy/interpolate/_interpolate.py:497: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/home/walkerdavis/projects/mpsc/.venv/lib/python3.12/site-packages/scipy/interpolate/_interpolate.py:497: RuntimeWarning: invalid value encountered in divide
  slop

In [19]:
consolidated = analysis.consolidate_all_data(
    demographics,
    energy_data,
    pulse_data,
    io_data,
    spo2_data,
    resp_data,
    stool_data,
    bp_data,
)
consolidated

/home/walkerdavis/projects/mpsc/.venv/lib/python3.12/site-packages/scipy/interpolate/_interpolate.py:497: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/home/walkerdavis/projects/mpsc/.venv/lib/python3.12/site-packages/scipy/interpolate/_interpolate.py:497: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]


,ID,DOB,GA,BW,Sex,StartDate,DailyEnergyEnteral,DailyEnergyParenteral,DailyEnergy,MeanDailyPulse,...,WeeklyWeightGainGPerKGPerDay,MeanDailyPulsePerKG,DailyFluidIntakePerKG,DailyFluidOutputPerKG,DailyFluidIntake_rolling_mean,DailyFluidIntake_rolling_std,DailyFluidOutput_rolling_mean,DailyFluidOutput_rolling_std,GrowthStage,Train
25258,Lurie_000008,2023-01-01,38.0,2620,M,2023-01-07,43.94,172.59,216.53,121.400000,...,-10.473205,38.035322,113.708069,41.982975,362.9300,NaN,134.000000,NaN,3,True
25259,Lurie_000008,2023-01-01,38.0,2620,M,2023-01-08,27.04,290.75,317.79,106.619048,...,-10.512859,33.757918,130.511519,140.580093,387.5650,34.839151,289.000000,219.203102,3,True
25260,Lurie_000008,2023-01-01,38.0,2620,M,2023-01-09,54.08,288.30,342.38,126.666667,...,-10.438579,40.514044,137.601686,166.960508,401.7800,34.829325,366.666667,205.234825,3,True
25261,Lurie_000008,2023-01-01,38.0,2620,M,2023-01-10,47.32,302.52,349.84,150.400000,...,-10.327781,48.562662,128.14537,87.341755,400.5525,28.543796,342.625000,174.335584,3,True
25262,Lurie_000008,2023-01-01,38.0,2620,M,2023-01-11,67.60,349.52,417.12,134.142857,...,-9.60178,43.721002,146.661379,178.41335,410.4380,33.161375,383.580000,176.582083,3,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12567,Lurie_001800,2024-01-01,23.0,700,M,2024-04-08,337.58,21.27,358.85,141.833333,...,NaN,NaN,NaN,NaN,90.5600,128.071180,251.900000,110.450079,3,True
15298,Lurie_001801,2025-01-01,39.0,3665,M,2025-01-02,0.00,82.35,82.35,90.333333,...,NaN,NaN,NaN,NaN,0.0000,NaN,64.000000,NaN,3,False
11961,Lurie_001802,2025-01-01,35.0,3970,F,2025-01-14,94.68,0.00,94.68,160.000000,...,NaN,NaN,NaN,NaN,221.5100,NaN,163.000000,NaN,3,True
11962,Lurie_001802,2025-01-01,35.0,3970,F,2025-01-15,407.13,0.00,407.13,166.333333,...,NaN,NaN,NaN,NaN,110.7550,156.631223,234.500000,101.116270,3,True


In [20]:
final_data = analysis.add_normal_growth_comparison()
final_data = analysis.create_polynomial_features()

final_data.to_csv(os.path.join(DATA_ROOT, "Consolidated.csv"), index=False)

final_data

/tmp/ipykernel_74276/3340171845.py:473: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.consolidated_data[f"{feature}_squared"] = (
/tmp/ipykernel_74276/3340171845.py:477: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.consolidated_data[f"{feature}_sqrt"] = np.where(
/tmp/ipykernel_74276/3340171845.py:473: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To

,ID,DOB,GA,BW,Sex,StartDate,DailyEnergyEnteral,DailyEnergyParenteral,DailyEnergy,MeanDailyPulse,...,DailyFluidOutputPerKG_sqrt_squared,DailyFluidOutputPerKG_sqrt_sqrt,DailyFluidOutput_rolling_mean_squared_squared,DailyFluidOutput_rolling_mean_squared_sqrt,DailyFluidOutput_rolling_mean_sqrt_squared,DailyFluidOutput_rolling_mean_sqrt_sqrt,DailyFluidOutput_rolling_std_squared_squared,DailyFluidOutput_rolling_std_squared_sqrt,DailyFluidOutput_rolling_std_sqrt_squared,DailyFluidOutput_rolling_std_sqrt_sqrt
25258,Lurie_000008,2023-01-01,38.0,2620,M,2023-01-07,43.94,172.59,216.53,121.400000,...,41.982975,2.545472,3.224179e+08,134.000000,134.000000,3.402328,NaN,NaN,NaN,NaN
25259,Lurie_000008,2023-01-01,38.0,2620,M,2023-01-08,27.04,290.75,317.79,106.619048,...,140.580093,3.443348,6.975757e+09,289.000000,289.000000,4.123106,2.308802e+09,219.203102,219.203102,3.847793
25260,Lurie_000008,2023-01-01,38.0,2620,M,2023-01-09,54.08,288.30,342.38,126.666667,...,166.960508,3.594624,1.807531e+10,366.666667,366.666667,4.375905,1.774207e+09,205.234825,205.234825,3.784973
25261,Lurie_000008,2023-01-01,38.0,2620,M,2023-01-10,47.32,302.52,349.84,150.400000,...,87.341755,3.057071,1.378086e+10,342.625000,342.625000,4.302340,9.237281e+08,174.335584,174.335584,3.633679
25262,Lurie_000008,2023-01-01,38.0,2620,M,2023-01-11,67.60,349.52,417.12,134.142857,...,178.41335,3.654743,2.164830e+10,383.580000,383.580000,4.425517,9.722692e+08,176.582083,176.582083,3.645328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12567,Lurie_001800,2024-01-01,23.0,700,M,2024-04-08,337.58,21.27,358.85,141.833333,...,NaN,NaN,4.026361e+09,251.900000,251.900000,3.983887,1.488210e+08,110.450079,110.450079,3.241839
15298,Lurie_001801,2025-01-01,39.0,3665,M,2025-01-02,0.00,82.35,82.35,90.333333,...,NaN,NaN,1.677722e+07,64.000000,64.000000,2.828427,NaN,NaN,NaN,NaN
11961,Lurie_001802,2025-01-01,35.0,3970,F,2025-01-14,94.68,0.00,94.68,160.000000,...,NaN,NaN,7.059118e+08,163.000000,163.000000,3.573114,NaN,NaN,NaN,NaN
11962,Lurie_001802,2025-01-01,35.0,3970,F,2025-01-15,407.13,0.00,407.13,166.333333,...,NaN,NaN,3.023928e+09,234.500000,234.500000,3.913233,1.045404e+08,101.116270,101.116270,3.171066
